In [1]:
import bempp.api
import numpy as np
import scipy 
import krypy
import time
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator
from scipy.sparse.linalg import LinearOperator

In [2]:
h = 0.05
r1 = 1
r2 = 1
dist = 3
sphere1 = bempp.api.shapes.sphere(r = r1, h=h, origin=((dist/2) + r1, 0, 0))
sphere2 = bempp.api.shapes.sphere(r = r2, h=h, origin=(-((dist/2) + r2), 0, 0))

In [3]:
sphere1.number_of_elements

12578

In [4]:
space1 = bempp.api.function_space(sphere1,'P',1)
space2 = bempp.api.function_space(sphere2,'P',1)

In [5]:
dim = 100

In [7]:
wavenumber = 0 * 1j

In [8]:
slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space1, space1, space1, wavenumber)
slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space2, space1, space1, wavenumber)
slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space1, space2, space2, wavenumber)
slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space2, space2, space2, wavenumber)

In [9]:
mat11 = slp11.weak_form().A
mat12 = slp12.weak_form().A
mat21 = slp21.weak_form().A
mat22 = slp22.weak_form().A

mat = np.block([[mat11,mat12],
                  [mat21,mat22]])  

L1, U1 = scipy.linalg.lu_factor(mat11)
L2, U2 = scipy.linalg.lu_factor(mat22)

In [10]:
def mv(v):
    return mat @ (list(scipy.linalg.lu_solve([L1, U1], v[0:mat11.shape[0]])) + list(scipy.linalg.lu_solve([L2, U2], v[mat11.shape[0]:])))

n = mat.shape[0]
x = (np.ones(n)/np.linalg.norm(np.ones(n))).reshape((n,1))
L_Op = LinearOperator(shape = (n,n) , matvec=mv)

In [14]:
V, H = krypy.utils.arnoldi(L_Op, x, maxiter = dim,  ortho='dmgs')
evals_H_, evect_H_ = np.linalg.eig(H[0:dim, :])  

In [15]:
logdet = 0
for eval_ in evals_H_:
    logdet += np.log(eval_)
print(logdet)

-0.0447580027569575


In [16]:
evals_H_

array([1.20864725, 0.79135275, 1.00908316, 1.0090832 , 1.00908318,
       0.9909168 , 0.99091682, 0.99091684, 1.00039543, 1.00039542,
       1.00039542, 1.00039542, 1.00039542, 0.99960457, 0.99960458,
       0.99960458, 0.99960458, 0.99960458, 0.99998279, 0.99998279,
       0.99998279, 1.00001721, 1.00001721, 1.00001721, 1.00001721,
       1.00001721, 1.00001721, 0.99998279, 0.99998279, 0.99998279,
       0.99998279, 1.00001721, 0.99999925, 0.99999925, 0.99999925,
       0.99999925, 1.00000075, 1.00000075, 1.00000075, 1.00000075,
       1.00000075, 1.00000075, 1.00000075, 0.99999925, 0.99999925,
       0.99999925, 1.00000075, 0.99999925, 0.99999925, 1.00000075,
       1.00000003, 1.00000003, 1.00000003, 0.99999997, 0.99999997,
       0.99999997, 0.99999997, 0.99999997, 0.99999997, 1.00000003,
       1.00000003, 1.00000003, 1.00000003, 1.00000003, 1.00000003,
       0.99999997, 0.99999997, 0.99999997, 0.99999997, 1.00000003,
       1.00000003, 0.99999997, 1.        , 1.        , 1.     

In [17]:
%store evals_H_

Stored 'evals_H_' (ndarray)
